# Generate report from exprted photometer data  

Use raw folder with photometer exported data

## Imports

In [ ]:
VERBOSE_NOTEBOOK = False
WARNING_DISABLE = True
DEBUG = False

In [ ]:
from os import path
import warnings
from scipy.optimize import OptimizeWarning

if WARNING_DISABLE:
    warnings.simplefilter('ignore', RuntimeWarning)
    warnings.simplefilter('ignore', OptimizeWarning)
    warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [ ]:
from mkinout import make_input_paths, parse_dir_name

# WORKING_DIR = './reports/export/230712_AAV8-ELISA_sey_GN004240-048'
WORKING_DIR = './reports/export/230801_AAV9-ELISA_sey_GN004240-053'
DATA_DIR = './data'

PARSED_DIR = parse_dir_name(WORKING_DIR)

input_files = make_input_paths(WORKING_DIR)
WORKLIST_FILE_PATH = input_files['worklist']
PARAMS_FILE_PATH = input_files['params']

## Layouts

In [ ]:
from readdata import read_layouts

PLATE_LAYOUT_ID = 'plate_layout_ident.csv'
PLATE_LAYOUT_NUM = 'plate_layout_num.csv'
PLATE_LAYOUT_DIL_ID = 'plate_layout_dil_id.csv'


g_lay = read_layouts(path.join(DATA_DIR, PLATE_LAYOUT_ID),
                     path.join(DATA_DIR, PLATE_LAYOUT_NUM),
                     path.join(DATA_DIR, PLATE_LAYOUT_DIL_ID))

if VERBOSE_NOTEBOOK:
    display(g_lay)

## Worklist

In [ ]:
from worklist import read_worklist, check_worklist
from readdata import read_params

g_wl_raw = read_worklist(WORKLIST_FILE_PATH)
g_params = read_params(PARAMS_FILE_PATH)

## Dilution to Concentration

Define dilution dataframe. The dataframe is indexed according plate layout, index of refference dataframe corresponds to refference of the `plate_layout_dil`.

In [ ]:
# TODO: read reference value from parameters
REF_VAL_MAX = 1.7954e+10
DILUTIONS = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0]

from sample import make_concentration
g_reference_conc = make_concentration(REF_VAL_MAX, DILUTIONS)

if VERBOSE_NOTEBOOK:
    display(g_reference_conc)

## Report generation

In [ ]:
from reportgen import gen_report_raw

reports = gen_report_raw(g_wl_raw, g_params, g_lay, g_reference_conc, WORKING_DIR)

In [ ]:
from mdhandling import export_palte_reports
from mdhandling import export_main_report

GENERATE_PLATE_PDF = True
GENERATE_PLATE_DOCX = False

export_main_report(reports, WORKING_DIR)
export_palte_reports(reports, GENERATE_PLATE_DOCX, GENERATE_PLATE_PDF)

In [ ]:
from reportmain import check_report_crc

CHECK_REPORT_CRC = True
REPORT_PLATES_CRC = [3422234280, 3350913816, 931962603]
if CHECK_REPORT_CRC:
    for report, crc in zip(reports, REPORT_PLATES_CRC):
        try:
            check_report_crc(report['md'], crc)
        except Exception as e:
            print('{} for {}'.format(e, report['path']))